In [1]:
import numpy as np
import pickle
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVR, SVC
from scipy.sparse import vstack, hstack, coo_matrix
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier

In [2]:
def msle(y_pred, y_true):
    first_log = np.log(np.clip(y_pred, np.finfo(float).eps, None) + 1.)
    second_log = np.log(y_true + 1.)
    return np.mean(np.square(first_log - second_log))

In [3]:
### FORTWNW KATEUTHIAN TO WORDS_LIST GIA NA MIN KANW LEMMATIZE KATHE FORA 12 LEPTA!!!!!!

with open('lemmatized_titles.pk', 'rb') as handle:
    words_list = pickle.load(handle)
len(words_list)  ###prepei na einai o arithmos twn arthrwn

51260

In [4]:
with open("article_id.pk", 'rb') as handle:
    article_id = pickle.load(handle).tolist()
len(article_id)

51260

In [5]:
with open('all_data_emb.pk', 'rb') as handle:
    data = pickle.load(handle)

with open('training_set.pickle', 'rb') as handle:
    train_set = pickle.load(handle)

with open('test_set.pickle', 'rb') as handle:
    test_set = pickle.load(handle)

In [6]:
text_tfidf = TfidfVectorizer(min_df=3, max_df=0.50, max_features=10000)
text_sparse = text_tfidf.fit_transform([' '.join(x) for x in words_list]).astype(np.float32)
np.save('tfidf_titles', text_sparse)

In [10]:
#NEW TRAIN/TEST SETS
import pandas as pd

week = []
l =['00','01','02','06','07','08']
for i in l:
    week.append(pd.read_csv('Access_Statistics/week'+i+'.txt', header = None, sep = '\t'))

In [44]:
###to train tha einai ta unique twn 2 prwtwn bdomadwn kai to test ta unique tis 3is...episis lambanw upopsin to
### impression mono tis prwti fora pou emfanizetai to arthro
split_train1 = {}

for w in week[:2]:    #2 prwtes bdomades(0,1)
    for i,k in zip(w[1],w[2]):     # 1:id , 2:impression
        if i not in split_train1:
            split_train1[i] = k

            
split_test1 = {}

for w in week[2:3]:    #3i bdomada (dld index=2)...sto indexing pairnei tin arxi kai afinei to telos eksw
     for i,k in zip(w[1],w[2]):
            if i not in split_test1 and i not in split_train1:
                split_test1[i] = k    # an den uparxei to dimiourgoume arxika
            

print(len(split_train1), len(split_test1))

27941 4366


In [22]:
### sunartisi arxikopoisis
def prepare_set_regression(set_dict):
    WC, A, P, Y = [], [], [], []  #WC= word codes, A= annotations, P= producers, Y= ta ids me ta impressions

    for key,value in set_dict.items():    #key=id value=impression  ,items() method is used to return the list with all dictionary keys with values
        if key in data:
            Y.append(value)         ##to value einai to impression kai to key einai to article id
            WC.append(text_sparse[article_id.index(key)])
            A.append(data[key]['annotations'])
            temp = np.zeros((5))
            temp[data[key]['publishers']] = 1
            P.append(temp)

    Y = np.array(Y)
    X = vstack(WC)
    A = coo_matrix(A).tocsr()
    P = coo_matrix(P).tocsr()
    X = hstack([X, A, P]).tocsr()

    print(Y.shape, X.shape, A.shape, P.shape)
    return Y, X

In [32]:
def prepare_set_classification(set_dict):
    WC, A, P, Y = [], [], [], []  #WC= word codes, A= annotations, P= producers, Y= ta ids me ta impressions

    for key,value in set_dict.items():    #key=id value=impression  ,items() method is used to return the list with all dictionary keys with values
        if key in data:
            if value<10:
                Y.append(0)         ##to value einai to impression kai to key einai to article id
            elif value<100:
                Y.append(1)
            elif value<1000:
                Y.append(2)
            else:
                Y.append(3)
            WC.append(text_sparse[article_id.index(key)])
            A.append(data[key]['annotations'])
            temp = np.zeros((5))
            temp[data[key]['publishers']] = 1
            P.append(temp)

    Y = np.array(Y)
    X = vstack(WC)
    A = coo_matrix(A).tocsr()
    P = coo_matrix(P).tocsr()
    X = hstack([X, A, P]).tocsr()

    print(Y.shape, X.shape, A.shape, P.shape)
    return Y, X

In [23]:
Y_train, X_train = prepare_set_regression(split_train1)
Y_test, X_test = prepare_set_regression(split_test1)

(31058,) (31058, 11003) (31058, 998) (31058, 5)
(4454,) (4454, 11003) (4454, 998) (4454, 5)


In [45]:
Y_train, X_train = prepare_set_classification(split_train1)
Y_test, X_test = prepare_set_classification(split_test1)

(27926,) (27926, 11003) (27926, 998) (27926, 5)
(4365,) (4365, 11003) (4365, 998) (4365, 5)


In [71]:
train_ids = np.where(Y_train>100)[0]
test_ids = np.where(Y_test>100)[0]
X_train = X_train[train_ids]
Y_train = Y_train[train_ids]
X_test = X_test[test_ids]
Y_test = Y_test[test_ids]
print(Y_test.shape, X_test.shape)

(3110,) (3110, 11003)


In [24]:
Y_pred = LinearRegression().fit(X_train, Y_train).predict(X_test)
msle(Y_pred, Y_test)

22.304624373185877

In [22]:
svr_rbf = SVR(kernel='rbf', gamma='scale', verbose=True)
Y_pred = svr_rbf.fit(X_train, Y_train).predict(X_test)
msle(Y_pred, Y_test)

[LibSVM]

5.50012823573697

In [27]:
size = Y_test.shape[0]*0.1
len(set(Y_test.argsort().tolist()[::-1][:int(size)]).intersection(set(Y_pred.argsort().tolist()[::-1][:int(size)]))) / size, size

(0.13920071845532106, 445.40000000000003)

In [28]:
from sklearn.metrics import average_precision_score

size = int(len(Y_test)*0.1)
print(size)
# threshold = -Y_test.sort()[:size]
threshold = np.min(np.sort(Y_test, axis=0)[::-1][:size])
y_true = Y_test > threshold

average_precision_score(y_true, Y_pred)

445


0.11631148949597167

In [52]:
logreg = LogisticRegression(solver='lbfgs', multi_class='multinomial', max_iter=1000)
Y_pred = logreg.fit(X_train, Y_train).predict(X_test)
np.mean(Y_pred == Y_test)

0.47949599083619704

In [48]:
svc_rbf = SVC(kernel='rbf', gamma='scale')
Y_pred = svc_rbf.fit(X_train, Y_train).predict(X_test)
np.mean(Y_pred == Y_test)

0.3214203894616266

In [47]:
clf = DecisionTreeClassifier(random_state=0)
Y_pred = clf.fit(X_train, Y_train).predict(X_test)
np.mean(Y_pred == Y_test)

0.4350515463917526